# Individual Project
By Jeff Akins

#### ![EV Charging Station](https://i.insider.com/6042a8ac44d8e300117b9655?width=1200&format=jpeg)

## Exploring the State of the Electric Vehicle Charging Network within the U.S.
The methods of transportation are changing, not only in the United States, but globally. The EU recently proposed a mandate that all new vehicles be emissions free after 2035. Here in the U.S., the Biden administrations wants half of all new vehicles to be purely electric by 2030. Regardless of the mandates, buyers may be choosing to purchase EVs over over their traditional internal combustion engine (ICE) counterparts in the upcoming years. No longer do you have to choose between the environment or the car that you want to drive. Many EVs are surpasing comparable ICE vehicles in both speed, handling, and overall capability and new EVs are being introduced on a regular basis. 

So, with all of these new EV coming online, is the charging infrastructure keeping up? 